In [1]:
import numpy as np
import pandas as pd
import theano
import theano.tensor as T

import lasagne

theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
path="data/"
data=pd.read_csv(path+"input_train.csv", index_col="ID")
X_ACC=data.filter(regex= "ACC*")
X_EEG=data.filter(regex= "EEG*")
labels=pd.read_csv(path+"challenge_output_data_training_file_sleep_stages_classification.csv", sep=";")

In [45]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
def create_input_features(X, n_channels, return_y=False):
    """
    takes values from one signal on n_channels and:
    - Divides it on train, validation and test
    - scales them
    - and formats them for use on the NN
    """
    X_train, X_test, y_train, y_test = train_test_split(X, labels["TARGET"].values, test_size=0.10, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    if return_y:
        return (
            X_train.reshape(X_train.shape[0],n_channels,X_train.shape[1]/n_channels), y_train,
            X_val.reshape(X_val.shape[0],n_channels,X_val.shape[1]/n_channels), y_val,
            X_test.reshape(X_test.shape[0],n_channels,X_test.shape[1]/n_channels), y_test     
        )  
    else:
        return (
            X_train.reshape(X_train.shape[0],n_channels,X_train.shape[1]/n_channels), 
            X_val.reshape(X_val.shape[0],n_channels,X_val.shape[1]/n_channels),
            X_test.reshape(X_test.shape[0],n_channels,X_test.shape[1]/n_channels)        
        )

In [48]:
X_ACC_train, X_ACC_val, X_ACC_test=create_input_features(X_ACC, 3)
X_EEG_train, y_train, X_EEG_val, y_val, X_EEG_test, y_test=create_input_features(X_EEG,1,True)

In [63]:
def build_cnn(input_var_EEG=None, input_var_ACC=None):
    #Two initialisation layers
    #One for EEG
    l_in1 = lasagne.layers.InputLayer(shape=(None,X_EEG_train.shape[1], X_EEG_train.shape[2]),
                                        input_var=input_var_EEG)
    #One for ACC
    l_in2 = lasagne.layers.InputLayer(shape=(None,X_ACC_train.shape[1], X_ACC_train.shape[2]),
                                        input_var=input_var_ACC)
    #Two convolutional layers to treat each signal separatedly
    l_conv1_1 = lasagne.layers.Conv1DLayer(
        l_in1, num_filters=8, filter_size=50,
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform())
    l_conv2_1 = lasagne.layers.Conv1DLayer(
        l_in2, num_filters=8, filter_size=5,
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform())
    #And again another 2 layers
    l_conv1_2 = lasagne.layers.Conv1DLayer(
        l_conv1_1, num_filters=8, filter_size=50,
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform())
    l_conv2_2 = lasagne.layers.Conv1DLayer(
        l_conv2_1, num_filters=8, filter_size=5,
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform())   
    #And two pooling layers
    l_pool1 = lasagne.layers.MaxPool1DLayer(l_conv1_2, pool_size=10)
    l_pool2 = lasagne.layers.MaxPool1DLayer(l_conv2_2, pool_size=1)

    #Two dense layers for each signal
    l_dense_1_1 = lasagne.layers.DenseLayer(
        lasagne.layers.dropout(l_pool1, p=.25),
        num_units=500,
        nonlinearity=lasagne.nonlinearities.rectify)
    l_dense_2_1 = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(l_pool2, p=.25),
        num_units=500,
        nonlinearity=lasagne.nonlinearities.rectify)
    #use neurons from both signals to predict
    #concatenate neurons
    l_concat = lasagne.layers.ConcatLayer([l_dense_1_1, l_dense_2_1], axis=1)
    #dense layer with all neurons
    l_dense_2 = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(l_concat, p=.25),
        num_units=1000,
        nonlinearity=lasagne.nonlinearities.rectify)
    #Output layer
    l_output = lasagne.layers.DenseLayer(
        lasagne.layers.dropout(l_dense_2, p=.25),
        num_units=5,
        nonlinearity=lasagne.nonlinearities.softmax)

    return l_output

In [22]:
#%pdb

In [64]:
# Prepare Theano variables for inputs and targets
input_var_EEG = T.tensor3('inputs')
input_var_ACC = T.tensor3('inputs')
target_var = T.ivector('targets')
# Create neural network model
network = build_cnn(input_var_EEG, input_var_ACC)

In [65]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.multiclass_hinge_loss(prediction, target_var)
loss = loss.mean()

In [66]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.0005, momentum=0.9)

In [67]:
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.multiclass_hinge_loss(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()

In [68]:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)

In [28]:
from tempfile import mkstemp

In [14]:
#tempfile.mkstemp(dir="../tmp")

In [15]:
#import os
#os.environ["THEANO_FLAGS"] = "blas.ldflags=-llapack -lblas -lgfortran"

In [69]:
train_fn =theano.function([input_var_EEG, input_var_ACC, target_var], loss, updates=updates)

In [70]:
val_fn = theano.function([input_var_EEG, input_var_ACC,target_var], [test_loss, test_acc])

In [71]:
def iterate_minibatches(input_1, input_2, targets, batchsize, shuffle=False):
    assert len(input_1) == len(targets)
    assert len(input_2) == len(targets)
    if shuffle:
        indices = np.arange(len(input_1))
        np.random.shuffle(indices)
    for start_idx in range(0, len(input_1) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield input_1[excerpt], input_2[excerpt], targets[excerpt]



In [ ]:
import time
num_epochs = 20
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_EEG_train, X_ACC_train, y_train, 64, shuffle=True):
        input_1, input_2, targets = batch
        targets = targets.astype(np.int32)
        train_err += train_fn(input_1, input_2, targets)
        train_batches += 1
        if train_batches % 10 == 0:
            print "epoch {}, train batches{}".format(epoch,train_batches) 

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_EEG_val, X_ACC_val, y_val, 64, shuffle=False):
        input_1, input_2, targets = batch
        targets = targets.astype(np.int32)
        err, acc = val_fn(input_1, input_2, targets)
        val_err += err
        val_acc += acc
        val_batches += 1
        if val_batches % 10 == 0:
            print "epoch {}, val batches{}".format(epoch,val_batches) 

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

epoch 0, train batches10
epoch 0, train batches20
epoch 0, train batches30
epoch 0, train batches40
epoch 0, train batches50
epoch 0, train batches60
epoch 0, train batches70
epoch 0, train batches80
epoch 0, train batches90
epoch 0, train batches100
epoch 0, train batches110
epoch 0, train batches120
epoch 0, train batches130
epoch 0, train batches140
epoch 0, train batches150
epoch 0, train batches160
epoch 0, train batches170
epoch 0, train batches180
epoch 0, train batches190
epoch 0, train batches200
epoch 0, train batches210
epoch 0, train batches220
epoch 0, train batches230
epoch 0, train batches240
epoch 0, train batches250
epoch 0, train batches260
epoch 0, train batches270
epoch 0, train batches280
epoch 0, train batches290
epoch 0, train batches300
epoch 0, train batches310
epoch 0, train batches320
epoch 0, train batches330
epoch 0, train batches340
epoch 0, train batches350
epoch 0, train batches360
epoch 0, train batches370
epoch 0, train batches380
epoch 0, train batche

In [26]:
from sklearn.metrics import accuracy_score
predicted_values = test_prediction.flatten()
print "test loss:", test_loss(predicted_values,y_test)

test loss:

TypeError: 'TensorVariable' object is not callable

In [27]:
predicted_values

Reshape{1}.0